# Demo of two-motor dynamic limit signal

## Overview

Some instrument designs need protection against accidental collision between moving parts during routine operations.  In such cases, each of the axes may be operating between its valid range but an interim (in-motion) state can pose the possibility of a collision.

One such possible collision is when arms of a diffractometer (such as $\theta$ and $2\theta$, known here as `theta` and `ttheta`, respectively) collide causing damage to beam transport apparatus and consequential instrumental downtime.

To prevent the collision *in this case*, the $2\theta$ axis must be at least $\delta$ degrees above the $\theta$ axis.  Empirically, $\delta$ of 3 degrees is sufficient protection.

From a controls safety view, we provide a signal that is zero when the move is not permitted and one when: ($2\theta - \theta) > \delta$

## EPICS setup

We start with two motor axes defined in EPICS.  Here, we run the docker image `prjemian/synApps-6.1` to make the [EPICS IOC simulator run in a docker container](https://github.com/prjemian/epics-docker/tree/master/n3_synApps#use-a-custom-ioc-prefix) with IOC prefix `sky:`.  These are motor PVs: `sky:m1` and `sky:m2` as shown.

![EPICS motor GUI screens](resources/th_tth_motors.png)

To compute a dynamic limit between the two motor axes, we use a *userCalc* (EPICS [swait]() record), `sky:userCalc1` with settings as shown.

1. Set the description field to describe what this does.
1. Monitor each motor's readback (`.RBV`) value.  The readback value is the motor record's *best* knowledge of the actual motor position.
1. `sky:userCalc1.INAN` = `sky:m1.RBV`, the value will be in `A` once the motor moves.
1. `sky:userCalc1.INBN` = `sky:m2.RBV`, the value will be in `B` once the motor moves.
1. Change the calculation's `.SCAN` field from *Passive* (calculates only when requested) to *I/O Intr* (calculate when any input changes, based on each field's *TRIGGER?* setting).
1. Enter the angle of minimum approach (3) into the `sky:userCalc1.C` field.  This will be the PV to change this number.
1. Enter the permit calculation: (B-A)>=C

The calculated result (in `sky:userCalc1.VAL`, `sky:userCalc1` for short) once either of the motors move.

![motion permit calculation](resources/th_tth_permit_calc.png)

To scan, we want a "detector".  Let's use another *userCalc* (`sky:userCalc2`) to simulate a noisy detector with a random number generator.  We'll update this detector only when either motor moves (same as with the permit calculation) setting its `.SCAN` to *I/O Intr*.  The setup is shown in the next screen view image:

![noisy detector simulation](resources/th_tth_permit_noisy.png)

## Bluesky setup

In [1]:
# short song-and-dance to locate my instrument package
import os, sys
path = os.path.join(
    os.environ["HOME"],
    "Documents",
    "projects",
    "instrument_poof",
)
sys.path.append(path)
del path

In [2]:
# we'll need these imports
from instrument.collection import *
from bluesky import plans as bp
from bluesky import plan_stubs as bps
from ophyd import Component, Device, EpicsMotor, EpicsSignal

I Tue-14:54:43 - ############################################################ startup
I Tue-14:54:43 - logging started
I Tue-14:54:43 - logging level = 10
I Tue-14:54:43 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/collection.py
I Tue-14:54:43 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/mpl/notebook.py


Activating auto-logging. Current session state plus future input saved.
Filename       : /home/prjemian/Documents/projects/use_bluesky/lessons/.logs/ipython_console.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


I Tue-14:54:43 - bluesky framework
I Tue-14:54:43 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/framework/check_python.py
I Tue-14:54:43 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/framework/check_bluesky.py
I Tue-14:54:44 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/framework/initialize.py
I Tue-14:54:45 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/framework/metadata.py
I Tue-14:54:45 - /home/prjemian/Documents/projects/use_bluesky/lessons/instrument/framework/callbacks.py
I Tue-14:54:45 - writing to SPEC file: /home/prjemian/Documents/projects/use_bluesky/lessons/20200901-145445.dat
I Tue-14:54:45 -    >>>>   Using default SPEC file name   <<<<
I Tue-14:54:45 -    file will be created when bluesky ends its next scan
I Tue-14:54:45 -    to change SPEC file, use command:   newSpecFile('title')


The active progress bar is great to indicate an activity in-progress but it interefers with the display in the jupyter notebook.  Disable the progress bar

In [3]:
RE.waiting_hook = None

symbol | PV (here) | meaning
--- | --- | ---
`theta` | sky:m1 | (`th`) EPICS motor record
`ttheta` | sky:m2 | (`tth`) EPICS motor record
`th_tth_permit` | sky:userCalc1.VAL | result of EPICS calculation (swait record: 1=permit, 0=not), updates when either motor moves
`th_tth_min` | sky:userCalc1.C | minimum permitted `\|th-tth\|`
`noisy` | sky:userCalc2.VAL | detector (random number generator) - integer, updates when either motor moves
`noisy_scale` | sky:userCalc2.C | scale factor for `noisy`

In [4]:
noisy = EpicsSignal("sky:userCalc2.VAL", name="noisy")
noisy_scale = EpicsSignal("sky:userCalc2.C", name="noisy_scale")
th_tth_min = EpicsSignal("sky:userCalc1.C", name="th_tth_min")
th_tth_permit = EpicsSignal("sky:userCalc1.VAL", name="th_tth_permit")
theta = EpicsMotor("sky:m1", name="theta", labels=["motors",])
ttheta = EpicsMotor("sky:m2", name="ttheta", labels=["motors",])

Define a plan for a coupled theta:2theta scan.

In [5]:
def th_tth_scan(detectors, tth_start, tth_end, points=11, min_sep=None):
    """
    run a coupled theta:2theta scan
    """
    min_sep = abs(min_sep or 3)
    old_sep = th_tth_min.get()

    # check end points first!
    if abs(tth_start/2) < min_sep:
        print(
            "Starting point below allowed minimum:"
            f" |{tth_start/2:.4f}| < |{min_sep:.4f}|")
        return
    if abs(tth_end/2) < min_sep:
        print(
            "Ending point below allowed minimum:"
            f" |{tth_end/2:.4f}| < |{min_sep:.4f}|")
        return

    yield from bps.mv(th_tth_min, min_sep)
    yield from bp.scan(
        detectors,
        theta, tth_start/2, tth_end/2,
        ttheta, tth_start, tth_end,
        points
    )

    # reset the previous minimum
    yield from bps.mv(th_tth_min, old_sep)

## Custom Bluesky plan for $\theta:2\theta$ scan

Try a scan that we know will fail the test for minimum separation.

In [6]:
RE(th_tth_scan([noisy, th_tth_permit], 5, 25, points=11, min_sep=3))

Starting point below allowed minimum: |2.5000| < |3.0000|


()

Swap the two end points, that also fails.

In [7]:
RE(th_tth_scan([noisy, th_tth_permit], 25, 5, points=11, min_sep=3))

Ending point below allowed minimum: |2.5000| < |3.0000|


()

This scan is successful.

In [8]:
RE(th_tth_scan([noisy, th_tth_permit], 9, 6, points=11, min_sep=3))



Transient Scan ID: 1     Time: 2020-09-01 14:55:17
Persistent Unique Scan ID: 'f1a98f6a-afea-4c15-baf8-0e4211eeb8ff'
New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+------------+------------+------------+---------------+
|   seq_num |       time |      theta |     ttheta |      noisy | th_tth_permit |
+-----------+------------+------------+------------+------------+---------------+
|         1 | 14:55:20.5 |    4.50000 |    9.00000 |      67219 |       1.00000 |
|         2 | 14:55:21.0 |    4.35000 |    8.70000 |      33101 |       1.00000 |
|         3 | 14:55:21.6 |    4.20000 |    8.40000 |        345 |       1.00000 |
|         4 | 14:55:22.2 |    4.05000 |    8.10000 |      92999 |       1.00000 |
|         5 | 14:55:22.8 |    3.90000 |    7.80000 |      14697 |       1.00000 |
|         6 | 14:55:23.4 |    3.75000 |    7.50000 |      52187 |       1.00000 |
|         7 | 14:55:24.0 |    3.60000 |    7.20000 |      77269 |       1.00000 |
|         8 | 14:55:24.6 |    3.45000 |    6.90000 |       3441 |       1.00000 |
|         9 | 14:55:25.2 |    3.30000 |    6.60000 |      28486 |       1.00000 |
|        10 | 14

('f1a98f6a-afea-4c15-baf8-0e4211eeb8ff',)

So far, have not registered a permit denied.  Have not encountered a condition where permit _would_ be denied.

## Try to provoke a permit denied

Check the backlash parameters for 2theta motor.  Since they are not part of `ophyd.EpicsMotor`, need to make a custom motor.

In [9]:
class MyMotor(EpicsMotor):
    backlash = Component(EpicsSignal, ".BDST")
    backlash_velocity = Component(EpicsSignal, ".BVEL")

# remake the two motors with backlash support
theta = MyMotor("sky:m1", name="theta", labels=["motors",])
ttheta = MyMotor("sky:m2", name="ttheta", labels=["motors",])

print(f"theta backlash distance: {theta.backlash.get()} ({theta.motor_egu.get()})")
print(f"theta backlash velocity: {theta.backlash_velocity.get()} ({theta.motor_egu.get()}/s)")

print(f"ttheta backlash distance: {ttheta.backlash.get()} ({ttheta.motor_egu.get()})")
print(f"ttheta backlash velocity: {ttheta.backlash_velocity.get()} ({ttheta.motor_egu.get()}/s)")

theta backlash distance: 0.0 (degrees)
theta backlash velocity: 1.0 (degrees/s)
ttheta backlash distance: 0.0 (degrees)
ttheta backlash velocity: 1.0 (degrees/s)


Reduce (just) the backlash velocity and set a backlash distance for `ttheta`.

In [10]:
%mov ttheta.backlash 0.5
%mov ttheta.backlash_velocity 0.2
print(f"ttheta backlash distance: {ttheta.backlash.get()} ({ttheta.motor_egu.get()})")
print(f"ttheta backlash velocity: {ttheta.backlash_velocity.get()} ({ttheta.motor_egu.get()}/s)")

ttheta backlash distance: 0.5 (degrees)
ttheta backlash velocity: 0.2 (degrees/s)


Scan again over a shorter range.

In [11]:
RE(th_tth_scan([noisy, th_tth_permit], 7, 6, points=4, min_sep=3))



Transient Scan ID: 2     Time: 2020-09-01 14:56:07
Persistent Unique Scan ID: '3315787d-d73f-4e35-8ba3-8c96e235ac9a'
New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+------------+------------+------------+---------------+
|   seq_num |       time |      theta |     ttheta |      noisy | th_tth_permit |
+-----------+------------+------------+------------+------------+---------------+
|         1 | 14:56:11.8 |    3.50000 |    7.00000 |      48504 |       1.00000 |
|         2 | 14:56:15.8 |    3.33000 |    6.67000 |      10437 |       1.00000 |
|         3 | 14:56:19.8 |    3.17000 |    6.33000 |      27610 |       1.00000 |
|         4 | 14:56:23.8 |    3.00000 |    6.00000 |      24640 |       1.00000 |
+-----------+------------+------------+------------+------------+---------------+
generator scan ['3315787d'] (scan num: 2)


('3315787d-d73f-4e35-8ba3-8c96e235ac9a',)

**Still**, the permit signal never shows a zero value because it is recorded at the **end** of each step.

Let's monitor the signal _during_ the scan.

In [12]:
sd.monitors.append(th_tth_permit)

Repeat the scan, collecting the new info.  We'll inspect the monitors after the scan is done.

In [13]:
uid = RE(th_tth_scan([noisy, th_tth_permit], 8, 6, points=4, min_sep=3))



Transient Scan ID: 3     Time: 2020-09-01 14:56:55
Persistent Unique Scan ID: '0a2b5391-78ac-4125-b01d-4095a4f85a23'
New stream: 'th_tth_permit_monitor'


<IPython.core.display.Javascript object>

New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+------------+------------+------------+---------------+
|   seq_num |       time |      theta |     ttheta |      noisy | th_tth_permit |
+-----------+------------+------------+------------+------------+---------------+
|         1 | 14:57:00.1 |    4.00000 |    8.00000 |      82319 |       1.00000 |
|         2 | 14:57:04.3 |    3.67000 |    7.33000 |      71824 |       1.00000 |
|         3 | 14:57:08.7 |    3.33000 |    6.67000 |      47439 |       1.00000 |
|         4 | 14:57:13.0 |    3.00000 |    6.00000 |      50977 |       1.00000 |
+-----------+------------+------------+------------+------------+---------------+
generator scan ['0a2b5391'] (scan num: 3)


Aha!  The first plot shows the monitored values.  It is clear the signal **does** go to 0 and then come back to 1.

Look at the monitored data.  First get the run object from databroker (indexed by the uid).  Then print the monitored data in a table.

In [14]:
run = db[uid[0]]
run.table("th_tth_permit_monitor")

,time,th_tth_permit
seq_num,,
2,2020-09-01 19:56:55.179922104,1.0
1,2020-09-01 19:56:55.179901361,1.0
3,2020-09-01 19:56:55.270913124,0.0
4,2020-09-01 19:56:55.273250341,1.0
5,2020-09-01 19:56:55.374645948,1.0
6,2020-09-01 19:56:55.378845215,1.0
7,2020-09-01 19:56:55.471202135,0.0
8,2020-09-01 19:56:55.473137856,1.0
9,2020-09-01 19:56:55.571295023,1.0


## Create a suspender

Block the RunEngine when the permit fails.

N.B. Might need to consider the special case where the permit fails when first starting the run.  Why did that fail? (We'll answer that soon.)

Following this example: https://blueskyproject.io/bluesky/state-machine.html#example-suspend-a-plan-if-the-beam-current-dips-low,
we'll do similar but know that our signal is a boolean, that indicates *no permit* when low.  Suspends interrupt the `RE` as long as the signal is invalid and automatically resume if the signal becomes valid again.  Let's see how this works here.

In [15]:
from bluesky.suspenders import SuspendBoolLow
sus = SuspendBoolLow(th_tth_permit)
RE.install_suspender(sus)

Repeat the scan.

In [16]:
uid = RE(th_tth_scan([noisy, th_tth_permit], 8, 6, points=4, min_sep=3))



Transient Scan ID: 4     Time: 2020-09-01 14:58:03
Persistent Unique Scan ID: 'f44305fd-2d1c-4f29-8737-fca2e5c55fd3'
New stream: 'th_tth_permit_monitor'


<IPython.core.display.Javascript object>

Suspending....To get prompt hit Ctrl-C twice to pause.
Suspension occurred at 2020-09-01 14:58:03.
Suspender SuspendBoolLow(EpicsSignal(read_pv='sky:userCalc1.VAL', name='th_tth_permit', value=1.0, timestamp=1598990283.342036, tolerance=1e-05, auto_monitor=False, string=False, write_pv='sky:userCalc1.VAL', limits=False, put_complete=False), sleep=0, pre_plan=None, post_plan=None,tripped_message=) reports a return to nominal conditions. Will sleep for 0 seconds and then release suspension at 2020-09-01 14:58:03.
Justification for this suspension:
Signal th_tth_permit is low


Run aborted
Traceback (most recent call last):
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/run_engine.py", line 1365, in _run
    msg = self._plan_stack[-1].send(resp)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 1307, in __call__
    return (yield from plan)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 1160, in baseline_wrapper
    return (yield from plan)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 803, in monitor_during_wrapper
    return (yield from plan2)
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 170, in plan_mutator
    raise ex
  File "/home/prjemian/Apps/anaconda/envs/bluesky_2020_9/lib/python3.8/site-packages/bluesky/preprocessors.py", line 1

FailedStatus: MoveStatus(done=True, pos=theta, elapsed=0.1, success=False, settle_time=0.0)

Failed at the very start of the scan with th=3.05 and tth=6.05.  We saw this in the previous scans, as well.  This is suspiciously close to the minimum separation and is likely because we moved `theta` *before* moving `ttheta`. 

The signal does not automatically clear since it is only computed when the motor readback value changes.  We can clear this by moving the `ttheta` motor manually.  If we change our plan logic to move `ttheta` first, we might avoid this altogether.

In [17]:
%mov ttheta 6.5

Suspender SuspendBoolLow(EpicsSignal(read_pv='sky:userCalc1.VAL', name='th_tth_permit', value=1.0, timestamp=1598990296.954188, tolerance=1e-05, auto_monitor=False, string=False, write_pv='sky:userCalc1.VAL', limits=False, put_complete=False), sleep=0, pre_plan=None, post_plan=None,tripped_message=) reports a return to nominal conditions. Will sleep for 0 seconds and then release suspension at 2020-09-01 14:58:16.
ttheta:   2%|▌                        | 0.01/0.45 [00:00<00:05, 11.53s/degrees]
ttheta:   4%|█                        | 0.02/0.45 [00:00<00:04, 10.77s/degrees]
ttheta:   9%|██▏                      | 0.04/0.45 [00:00<00:03,  7.89s/degrees]
ttheta:  13%|███▎                     | 0.06/0.45 [00:00<00:02,  6.92s/degrees]
ttheta:  18%|████▍                    | 0.08/0.45 [00:00<00:02,  6.45s/degrees]
ttheta:  22%|█████▊                    | 0.1/0.45 [00:00<00:02,  6.16s/degrees]
ttheta:  27%|██████▋                  | 0.12/0.45 [00:00<00:01,  5.97s/degrees]
ttheta:  31%|███████▊ 

In [18]:
uid = RE(th_tth_scan([noisy, th_tth_permit], 8, 6, points=4, min_sep=3))



Transient Scan ID: 5     Time: 2020-09-01 14:58:34
Persistent Unique Scan ID: '0ba26227-09fc-4fa0-b871-757428943d8c'
New stream: 'th_tth_permit_monitor'
New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+------------+------------+------------+---------------+
|   seq_num |       time |      theta |     ttheta |      noisy | th_tth_permit |
+-----------+------------+------------+------------+------------+---------------+
|         1 | 14:58:38.9 |    4.00000 |    8.00000 |      91188 |       1.00000 |
|         2 | 14:58:43.1 |    3.67000 |    7.33000 |      12869 |       1.00000 |
Suspending....To get prompt hit Ctrl-C twice to pause.
Suspension occurred at 2020-09-01 14:58:44.
Justification for this suspension:
Signal th_tth_permit is low
Suspender SuspendBoolLow(EpicsSignal(read_pv='sky:userCalc1.VAL', name='th_tth_permit', value=1.0, timestamp=1598990353.311899, tolerance=1e-05, auto_monitor=False, string=False, write_pv='sky:userCalc1.VAL', limits=False, put_complete=False), sleep=0, pre_plan=None, post_plan=None,tripped_message=) reports a return to nominal conditions. Will sleep for 0 seconds and then release suspension at 2020-09-01 14:59:13.
|         

Here it failed when `theta=3.33` and `ttheta=6.17` and the command line is not responsive.  Move `ttheta` from the GUI in another window.

again, as needed to complete the scan...

Still, the scan would not complete because the `RE` kept re-winding to the last move command which triggered the failiure again.  Had to override the calculation (temporarily) to allow the scan to complete.

Clearly, this idea needs a bit more work.  This suspender does its job but this is not a good fit for our problem.  **Or** we must be careful not to encounter this by considering the backlash motion is part of the scan.  Hard to capture that when it is a dynamic problem.

## Custom suspender

We might need to write our own suspender if none of the [provided suspenders](https://blueskyproject.io/bluesky/state-machine.html#built-in-suspenders) will do the job we want.  There is [an example](https://github.com/BCDA-APS/apstools/blob/master/apstools/suspenders.py#L25) of a custom suspender in [apstools](https://apstools.readthedocs.io).